# Launch Trelliscope Viewer for Testing

This notebook launches the existing **notebook_demo** display at http://localhost:8762.

**Use this to test the viewer without regenerating the display.**

## What This Does

1. ✅ Checks if the display exists
2. ✅ Kills any existing server on port 8762
3. ✅ Starts HTTP server from correct directory
4. ✅ Opens browser to the viewer
5. ✅ Shows server logs

In [1]:
import subprocess
import time
import webbrowser
from pathlib import Path
import os
import signal

print("Trelliscope Viewer Launcher")
print("="*70)

Trelliscope Viewer Launcher


## 1. Check Display Exists

In [2]:
# Display location
display_dir = Path("output/notebook_demo")

print(f"\nChecking display at: {display_dir.absolute()}")
print("-"*70)

# Check required files
required_files = [
    display_dir / "index.html",
    display_dir / "config.json",
    display_dir / "displays" / "displayList.json",
    display_dir / "displays" / "notebook_demo" / "displayInfo.json",
    display_dir / "displays" / "notebook_demo" / "metaData.js",
]

all_exist = True
for file_path in required_files:
    exists = file_path.exists()
    status = "✓" if exists else "✗"
    print(f"  {status} {file_path.relative_to(display_dir)}")
    if not exists:
        all_exist = False

# Check panels
panels_dir = display_dir / "displays" / "notebook_demo" / "panels"
panel_count = len(list(panels_dir.glob("*.png"))) if panels_dir.exists() else 0
print(f"  ✓ {panel_count} panels in displays/notebook_demo/panels/")

if not all_exist:
    print("\n❌ ERROR: Display files are missing!")
    print("\nRun the notebook '11_working_viewer_demo.ipynb' first to create the display.")
else:
    print("\n✅ All display files found!")


Checking display at: /Users/matthewdeane/Documents/Data Science/python/_projects/py-trelliscope2/examples/output/notebook_demo
----------------------------------------------------------------------
  ✓ index.html
  ✓ config.json
  ✓ displays/displayList.json
  ✓ displays/notebook_demo/displayInfo.json
  ✓ displays/notebook_demo/metaData.js
  ✓ 5 panels in displays/notebook_demo/panels/

✅ All display files found!


## 2. Kill Existing Server (if any)

In [3]:
port = 8762

print(f"\nChecking port {port}...")
print("-"*70)

# Check if port is in use
try:
    result = subprocess.run(
        ["lsof", "-ti", f":{port}"],
        capture_output=True,
        text=True
    )
    
    if result.stdout.strip():
        pids = result.stdout.strip().split()
        print(f"⚠️  Port {port} is in use by process(es): {', '.join(pids)}")
        
        # Kill the processes
        for pid in pids:
            try:
                os.kill(int(pid), signal.SIGTERM)
                print(f"  ✓ Killed process {pid}")
            except ProcessLookupError:
                print(f"  ℹ️  Process {pid} already stopped")
            except PermissionError:
                print(f"  ✗ No permission to kill process {pid}")
        
        time.sleep(1)  # Give processes time to clean up
        print(f"\n✅ Port {port} cleared")
    else:
        print(f"✅ Port {port} is available")
except FileNotFoundError:
    print("ℹ️  lsof command not available (Windows?)")
    print(f"  Assuming port {port} is available")
except Exception as e:
    print(f"⚠️  Could not check port: {e}")
    print(f"  Will try to start server anyway")


Checking port 8762...
----------------------------------------------------------------------
⚠️  Port 8762 is in use by process(es): 23001
  ✓ Killed process 23001

✅ Port 8762 cleared


## 3. Start HTTP Server

In [4]:
print(f"\nStarting HTTP server...")
print("-"*70)

# Start server from display root directory
server_process = subprocess.Popen(
    ["python3", "-m", "http.server", str(port)],
    cwd=display_dir,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

print(f"✓ Server started (PID: {server_process.pid})")
print(f"  Serving from: {display_dir.absolute()}")
print(f"  Port: {port}")

# Wait for server to start
time.sleep(2)

# Check if server is running
if server_process.poll() is None:
    print(f"\n✅ Server is running!")
else:
    print(f"\n❌ Server failed to start")
    stderr = server_process.stderr.read()
    if stderr:
        print(f"Error: {stderr}")


Starting HTTP server...
----------------------------------------------------------------------
✓ Server started (PID: 29206)
  Serving from: /Users/matthewdeane/Documents/Data Science/python/_projects/py-trelliscope2/examples/output/notebook_demo
  Port: 8762

✅ Server is running!


## 4. Open Browser

In [5]:
url = f"http://localhost:{port}/"

print(f"\nOpening browser...")
print("-"*70)

try:
    webbrowser.open(url)
    print(f"✓ Browser opened to: {url}")
except Exception as e:
    print(f"⚠️  Could not open browser: {e}")
    print(f"\nManually open: {url}")

print(f"\n{'='*70}")
print(f"🌐 VIEWER URL: {url}")
print(f"{'='*70}")
print(f"\nExpected Result:")
print(f"  ✓ 5 panels displayed in 3-column grid")
print(f"  ✓ Bar charts for categories A, B, C, D, E")
print(f"  ✓ Filter dropdown works")
print(f"  ✓ Sort by value works")
print(f"  ✓ Labels show under each panel")
print(f"\nServer is running in background.")
print(f"Server PID: {server_process.pid}")


Opening browser...
----------------------------------------------------------------------
✓ Browser opened to: http://localhost:8762/

🌐 VIEWER URL: http://localhost:8762/

Expected Result:
  ✓ 5 panels displayed in 3-column grid
  ✓ Bar charts for categories A, B, C, D, E
  ✓ Filter dropdown works
  ✓ Sort by value works
  ✓ Labels show under each panel

Server is running in background.
Server PID: 29206


## 5. Server Management

### To Stop the Server

Run the cell below to stop the server when you're done.

In [6]:
# Stop the server
try:
    if 'server_process' in globals() and server_process.poll() is None:
        server_process.terminate()
        server_process.wait(timeout=5)
        print(f"✓ Server stopped (PID: {server_process.pid})")
    else:
        print("ℹ️  Server is not running")
except Exception as e:
    print(f"⚠️  Error stopping server: {e}")
    print(f"\nManually kill the process:")
    print(f"  kill {server_process.pid}")

✓ Server stopped (PID: 29206)


### To Check Server Status

In [7]:
# Check if server is still running
if 'server_process' in globals():
    if server_process.poll() is None:
        print(f"✓ Server is running (PID: {server_process.pid})")
        print(f"  URL: http://localhost:{port}/")
    else:
        print(f"✗ Server has stopped")
        exit_code = server_process.returncode
        print(f"  Exit code: {exit_code}")
else:
    print("ℹ️  Server has not been started yet")
    print("  Run the cells above to start the server")

✗ Server has stopped
  Exit code: -15


## Troubleshooting

### If the viewer shows "0 of 0 panels"

1. **Check browser console** (F12 → Console tab)
   - Look for JavaScript errors
   - Check for 404 errors loading files

2. **Verify files exist** - Run cell 3 above

3. **Check server is running from correct directory**
   - Should be serving from `output/notebook_demo/`
   - NOT from `output/notebook_demo/displays/`

4. **Regenerate display** - Run `11_working_viewer_demo.ipynb`

### If port 8762 is in use

Run this in a terminal:
```bash
# Find process using port 8762
lsof -i :8762

# Kill the process
lsof -ti :8762 | xargs kill -9
```

Or change the port in cell 5 above.

### If browser doesn't open automatically

Manually open: http://localhost:8762/

### To restart the server

1. Run cell 11 (Stop Server)
2. Run cells 5, 7, 9 (Kill existing → Start → Open browser)

## Quick Reference

### File Structure

```
output/notebook_demo/              ← Server serves from here
├── index.html                     ← Entry point
├── config.json                    ← Viewer loads this first
└── displays/
    ├── displayList.json           ← List of displays
    └── notebook_demo/
        ├── displayInfo.json       ← Display config
        ├── metaData.js            ← Panel metadata (loaded by viewer)
        ├── metaData.json          ← Panel metadata (backup)
        └── panels/
            ├── 0.png              ← Panel images
            ├── 1.png
            └── ...
```

### Working Viewers

| Port | Display | Status |
|------|---------|--------|
| 9000 | simple_static | ✅ Reference |
| 8001 | test_static | ✅ Reference |
| **8762** | **notebook_demo** | ✅ **This notebook** |

### Key Commands

```bash
# Start server manually
cd output/notebook_demo
python3 -m http.server 8762

# Check what's using port
lsof -i :8762

# Kill server
kill <PID>
```